In [ ]:
# For feedback or questions contact us at: github@eset.com
# https://github.com/eset/eti/
#
# Author: ESET Research
#
# This code is provided to the community under the two-clause BSD license as
# follows:
#
# Copyright (C) 2025 ESET
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
# list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice,
# this list of conditions and the following disclaimer in the documentation
# and/or other materials provided with the distribution.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# Initializing your environment

## Installation as PyMISP user

The quick and dirty way:

```bash
sudo pip3 install pymisp
```

The clean approach as user:

```bash
pip3 install --user pymisp
```
## Setting up of jupyter

**We assume you're in a virtual environment**

If you want to follow along this workshop on your computer, this is the way to go:

```bash
pip3 install jupyter
jupyter-notebook
```

# Using the PyMISP objects

This page aims to give recommendations about how to efficiently use the `pymisp` library.

It is strongly recommended (read "don't do anything else, please") to use the library this way and never, ever modify the python dictionary you get by loading the JSON blob you receive from the server.

This library is made in a way to hide as much as the complexity as possible and we're happy to improve it is there is something missing.

## Interacting with a MISP instance

In [ ]:
import os
from pymisp import PyMISP

# Use var env to retrieve MISP URL and API key, if not you can provide a default one
MISP_URL: str = os.getenv("MISP_URL", "https://misp.eset.com")
MISP_API_KEY: str = os.getenv("MISP_API_KEY", "")

misp: PyMISP = PyMISP(MISP_URL, MISP_API_KEY, ssl=True)

### Fetching an event
Once the event is retrieved, we have several methods available to extract the information we need. For a complete list of these methods, refer to the documentation.
In MISP, objects are used to group related attributes in a structured and meaningful way. These objects are based on predefined templates that are bundled within the library.

  > NOTE: We recommend setting the parameter `pythonify=True` when interacting with PyMISP objects. This enables more intuitive access to object properties and methods.

#### Using event ID

In [ ]:
from pymisp import MISPEvent

event_id: int = 1523

if isinstance(event_id, int):
    event: MISPEvent = misp.get_event(event_id, pythonify=True)

if isinstance(event, MISPEvent):
    # Print the name of the event
    print(event.info)

#### Using event UUID

In [ ]:
from pymisp import MISPEvent
from uuid import UUID

event_uuid: UUID = UUID("00000000-0000-0000-0000-000000000000")

event: MISPEvent = misp.get_event(event_uuid, pythonify=True)
if isinstance(event, MISPEvent):
    # Print the name of the event
    print(event.info)

### Domains and IP addresses
This snippet shows how to retrieve domains and IP addresses.

In [ ]:
for obj in event.Object:
    if obj.name == "domain-ip":
        for attr in obj.Attribute:
            if attr.object_relation == "ip":
                print(f"[+] Found IP address: {attr.value}")
            elif attr.object_relation == "domain":
                print(f"[+] Found domain: {attr.value}")
            else:
                pass

## Victims
The MISPObject `victim` has the UUID defined here: https://github.com/MISP/misp-objects/blob/a6a5ecc1cbc3cfa0fcf76042510a26fc24b3aae7/objects/victim/definition.json#L139C12-L139C48

In [ ]:
import json

victim_object_uuid = "a8806e40-39ad-435f-be02-ac2a13d6fc7d"
victim_object: dict = misp.get_object_template(victim_object_uuid)
print(json.dumps(victim_object, indent=2))

### Sectors

This is an example of how to retrieve events where the sector targeted contains `transportation`.
The full list of the sector can be found via the link below.

In [ ]:
for attr in victim_object.get("ObjectTemplateElement"):
    if attr.get("object_relation") == "sectors":
        sector_list = attr.get("sane_default")
        break
print(f"Supported list of sector:\n* {"\n* ".join(sector_list)}")

Based on the list above, let's search for `transportation` sector.
  > WARNING: The search below is limited to the last 30 days, keep in mind that the more precise is your search the faster is the answer

In [ ]:
events: list[MISPEvent] = misp.search(
    publish_timestamp='30d'
    value="transportation",
    object_name="victim",
    published = True,
    pythonify=True
)

if isinstance(events, list):
    print(f"[+] Found {len(events)} events published during the last 30 days where the targeted sector contains `transportation`" )
    for event in events:
        print(event.info)

### Regions

The list of regions or locations from the victim targeted is using ISO 3166.
  > TODO: check for the broader region list

In [ ]:
events: list[MISPEvent] = misp.search(
    publish_timestamp='30d',
    value="SK",
    object_name="victim",
    published = True,
    pythonify=True
)

if isinstance(events, list):
    print(f"[+] Found {len(events)} events published during the last 30 days where the targeted region contains `SK / Slovakia`" )
    for event in events:
        print(event.info)

## Threat Actors

Here are a couple of examples how to search for specific Threat Actors and their characteristics

### ESET Threat Actors country of origin

Let's retrieve events tagged with threat actors' country of origin is `RU` (Russia).

There are a couple of steps to do that:
- retrieve ESET Threat Actors MISPGalaxy
- iterates through every cluster to retrieve the country of origin
- create a list of matching clusters
- search events tagged with those clusters

In [ ]:
from pymisp import MISPGalaxy

ESET_THREAT_ACTOR_GALAXY_UUID = "b9e6761a-d501-4cac-94da-8e75e5d26ddb"

# Retrieve ESET Threat Actors galaxy
galaxy: MISPGalaxy = misp.get_galaxy(
    ESET_THREAT_ACTOR_GALAXY_UUID,
    withCluster=True,
    pythonify=True
)

In [ ]:
# Create a list of Threat Actors where the country of origin matches the value `country_origin`
country_origin = "RU"
ta_list: list = []

for cluster in galaxy.clusters:
    for k, v in cluster.elements_meta.items():
        if k == "country":
            if len(v):
                country = v.pop()
                if country == country_origin:
                    ta_list.append(cluster)
                break
            elif len(v) > 1:
                print("[-] Multiple regions found for the same Threat Actor cluster.")
            else:
                print(f"[+] No region found for cluster {cluster.value}")
print(f"[+] Found {len(ta_list)} Threat Actors where the country of origin is {country_origin}.")

In [ ]:
# Search for events for the past 30 days matching our Threat Actors' list
events: list[MISPEvent] = misp.search(
    publish_timestamp='30d',
    event_tags=[i.tag_name for i in ta_list],
    published = True,
    pythonify=True
)

print(f"[+] Found {len(events)} events where the treat actors country of origin is {country_origin} during the last 30 days.")

### ESET Threat Actors alternative names

Different companies have different names for the same Threat Actor. When possible, we try to keep an up-to-date list of synonyms such that you can search for alternative names.

In [ ]:
from pymisp import MISPGalaxyCluster

ESET_THREAT_ACTOR_GALAXY_UUID = "b9e6761a-d501-4cac-94da-8e75e5d26ddb"

searching_ta = "APT41"

clusters: list[MISPGalaxyCluster] = misp.search_galaxy_clusters(
    ESET_THREAT_ACTOR_GALAXY_UUID,
    searchall=searching_ta,
    pythonify=True,
)

if isinstance(events, list):
    print(f"[+] Found {len(clusters)} cluster(s) related to {searching_ta}")
    for cluster in clusters:
        cluster_full = misp.get_galaxy_cluster(cluster, pythonify=True)
        print(cluster_full)